# Análise de Resultados: C-SILSP (PLIM vs. GRASP)

Este notebook implementa a metodologia de avaliação descrita no relatório do projeto. Vamos carregar os dados brutos dos arquivos `resultados_mip.csv` (Gurobi/PLIM) e `resultados_grasp.csv` (GRASP) para gerar as análises de:

1.  **Qualidade da Solução (Performance Profiles)**
2.  **Escalabilidade (Tempo e Qualidade vs. Horizonte $T$)**

Os **Time-to-Target (TTT) Plots** não serão gerados, pois os scripts atuais (`grasp_csilsp.py`) não foram configurados para logar o histórico de convergência (i.e., (tempo, melhor_custo) ao longo da execução). Seria necessário modificar o *solver* para salvar essa informação.

## 1. Setup: Importação e Carregamento dos Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Bibliotecas importadas com sucesso.")

In [ ]:
# Caminhos para os arquivos (ajuste se o notebook não estiver no mesmo diretório)
path_mip = 'benchmark/resultados_mip.csv'
path_grasp = 'benchmark/resultados_grasp.csv'

try:
    df_mip = pd.read_csv(path_mip)
    df_grasp = pd.read_csv(path_grasp)
    print(f"Carregado: {path_mip} ({len(df_mip)} linhas)")
    print(f"Carregado: {path_grasp} ({len(df_grasp)} linhas)")
except FileNotFoundError:
    print("Erro: Arquivos CSV não encontrados. Verifique os caminhos.")

print("\n--- Head Gurobi (MIP) ---")
print(df_mip.head())
print("\n--- Head GRASP ---")
print(df_grasp.head())

## 2. Limpeza e Fusão dos Dados

Para comparar os dois *solvers*, precisamos:
1.  Fazer um `merge` das tabelas usando as colunas-chave da instância (`classe`, `arquivo`, `T`, `tau`, `var`).
2.  Filtrar instâncias `INFEASIBLE` (infactíveis), que não entram na análise de performance.
3.  Converter colunas de custo para numérico, tratando valores nulos (ex: MIP sem solução no `TIME_LIMIT`).
4.  Usar `np.inf` para custos de *solvers* que falharam em encontrar uma solução factível.

In [ ]:
# Chaves para o merge
key_cols = ['classe', 'arquivo', 'T', 'tau', 'var']

# Merge dos dataframes
df = pd.merge(df_mip, df_grasp, on=key_cols, suffixes=('_mip', '_grasp'))

print(f"Linhas antes da filtragem: {len(df)}")

# 1. Filtrar instâncias infactíveis (onde o MIP provou infactibilidade)
df_clean = df[df['status'] != 'INFEASIBLE'].copy()
# Também removemos as que o GRASP marcou como infactível (factivel_grasp == 0)
df_clean = df_clean[df_clean['factivel_grasp'] == 1]

print(f"Linhas após filtrar infactíveis: {len(df_clean)}")

# 2. Converter custos e bounds para numérico, tratando erros
# (Gurobi pode ter 'None' ou '' se não achar solução)
df_clean['custo_mip'] = pd.to_numeric(df_clean['custo_mip'], errors='coerce')
df_clean['bound_mip'] = pd.to_numeric(df_clean['bound_mip'], errors='coerce')
df_clean['custo_grasp'] = pd.to_numeric(df_clean['custo_grasp'], errors='coerce') # Já deve ser numérico, mas é seguro

# 3. Preencher custos não encontrados (ex: TIME_LIMIT sem solução) com Infinito
df_clean['custo_mip'] = df_clean['custo_mip'].fillna(np.inf)
df_clean['custo_grasp'] = df_clean['custo_grasp'].fillna(np.inf) # GRASP não deve falhar, mas é boa prática

# 4. Preencher bounds não encontrados (ex: falha) com -Inf (para gaps)
df_clean['bound_mip'] = df_clean['bound_mip'].fillna(-np.inf)

print("\nDados limpos e fundidos. Colunas de custo prontas para análise.")
df_clean.info()

## 3. Análise de Qualidade (Performance Profiles)

O *Performance Profile* (Dolan & Moré, 2002) compara a qualidade dos *solvers*.

1.  Para cada instância $p$, encontramos o **melhor custo** $C_p^* = \min(C_{p, mip}, C_{p, grasp})$.
2.  Calculamos o **fator de performance** $r_{p,s}$ para cada *solver* $s$: $r_{p,s} = C_{p,s} / C_p^*$.
3.  O gráfico mostra $P(r)$, a proporção de instâncias que o *solver* $s$ resolveu com um custo $\le r \times C_p^*$.

In [ ]:
# 1. Encontrar o melhor custo (somente de soluções factíveis encontradas)
df_perf = df_clean[df_clean['custo_mip'] < np.inf].copy() # Ignora falhas do Gurobi por enquanto
df_perf = df_perf[df_perf['custo_grasp'] < np.inf] # Ignora falhas do GRASP

if len(df_perf) > 0:
    df_perf['best_cost'] = df_perf[['custo_mip', 'custo_grasp']].min(axis=1)
    
    # 2. Calcular os fatores de performance
    # Adicionamos 'epsilon' para evitar divisão por zero se best_cost for 0
    epsilon = 1e-9
    df_perf['r_mip'] = df_perf['custo_mip'] / (df_perf['best_cost'] + epsilon)
    df_perf['r_grasp'] = df_perf['custo_grasp'] / (df_perf['best_cost'] + epsilon)

    # 3. Preparar dados para o gráfico (Plotando a ECDF - Empirical CDF)
    
    # Eixo x (fatores de performance)
    # Queremos ver de r=1 até r=2 (ou o máximo, o que for menor)
    max_ratio = max(df_perf['r_mip'].max(), df_perf['r_grasp'].max())
    plot_max_r = min(max_ratio, 2.0) 
    r_axis = np.linspace(1, plot_max_r, 200)

    # Eixo y (Probabilidade P(r))
    n_problems = len(df_perf)
    cdf_mip = []
    cdf_grasp = []
    
    for r in r_axis:
        cdf_mip.append( (df_perf['r_mip'] <= r).sum() / n_problems )
        cdf_grasp.append( (df_perf['r_grasp'] <= r).sum() / n_problems )

    # 4. Plotar
    plt.figure(figsize=(12, 7))
    plt.plot(r_axis, cdf_mip, label='Gurobi (PLIM)', lw=2.5, marker='s', markevery=15, ms=6)
    plt.plot(r_axis, cdf_grasp, label='GRASP', lw=2.5, marker='o', markevery=15, ms=6)
    
    plt.title('Performance Profile (Qualidade da Solução)', fontsize=16)
    plt.xlabel('Fator de Performance (r)', fontsize=14)
    plt.ylabel('Proporção de Instâncias Resolvidas $P(r)$', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, which='both', linestyle='--', alpha=0.7)
    plt.xlim(1, plot_max_r)
    plt.ylim(0, 1.05)
    plt.show()
else:
    print("Nenhuma instância factível encontrada por ambos os solvers para plotar o P-Profile.")


## 4. Análise de Escalabilidade

Aqui, avaliamos como o desempenho (tempo e qualidade) se degrada com o aumento do horizonte $T$.

### 4.1. Escalabilidade do Tempo (Gurobi)

Vamos focar no tempo que o Gurobi leva para provar a **otimalidade**, pois o tempo do GRASP é fixo (limitado pelo `time_limit`).

In [ ]:
# Filtra apenas instâncias onde Gurobi atingiu o ótimo
df_optimal = df_clean[df_clean['status'] == 'OPTIMAL'].copy()

if len(df_optimal) > 0:
    plt.figure(figsize=(10, 6))
    # Usamos 'T' como uma categoria (string) para o boxplot tratar T=50 e T=100 igualmente
    df_optimal['T_cat'] = df_optimal['T'].astype(str)
    
    sns.boxplot(data=df_optimal, x='T_cat', y='tempo_seg_mip', order=sorted(df_optimal['T_cat'].unique()))
    
    plt.title('Escalabilidade Gurobi: Tempo para Provar Otimalidade', fontsize=16)
    plt.xlabel('Horizonte de Planejamento (T)', fontsize=14)
    plt.ylabel('Tempo (segundos) [Escala Log]', fontsize=14)
    plt.yscale('log') # O tempo de prova cresce exponencialmente, log é essencial
    plt.show()
else:
    print("Nenhuma instância resolvida até a otimalidade pelo Gurobi para analisar tempo.")

### 4.2. Escalabilidade da Qualidade (GRASP)

Como a qualidade da solução do GRASP (medida pelo *gap* de otimalidade) se comporta à medida que $T$ aumenta?

O *gap* é calculado em relação ao *best bound* (limite dual) fornecido pelo Gurobi, que é a melhor "garantia" de qualidade que temos.

$\text{Gap (\%)} = 100 \times \frac{\text{Custo}_{GRASP} - \text{Bound}_{MIP}}{\text{Custo}_{GRASP}}$ (conforme relatório)

In [ ]:
# Usamos o df_perf, que contém instâncias factíveis e bounds válidos
df_gap_analysis = df_perf[df_perf['bound_mip'] > 0].copy() # Filtra bounds inválidos

if len(df_gap_analysis) > 0:
    # Calcular o gap percentual (conforme fórmula do relatório)
    df_gap_analysis['gap_grasp_pct'] = 100 * (df_gap_analysis['custo_grasp'] - df_gap_analysis['bound_mip']) / df_gap_analysis['custo_grasp']
    
    # Tratar casos onde o bound pode ser ligeiramente maior que o custo (erros de precisão)
    df_gap_analysis['gap_grasp_pct'] = df_gap_analysis['gap_grasp_pct'].clip(lower=0)
    
    plt.figure(figsize=(10, 6))
    df_gap_analysis['T_cat'] = df_gap_analysis['T'].astype(str)
    
    sns.boxplot(data=df_gap_analysis, x='T_cat', y='gap_grasp_pct', order=sorted(df_gap_analysis['T_cat'].unique()))
    
    plt.title('Escalabilidade GRASP: Qualidade da Solução (Gap)', fontsize=16)
    plt.xlabel('Horizonte de Planejamento (T)', fontsize=14)
    plt.ylabel('Gap de Otimalidade do GRASP (%)', fontsize=14)
    plt.ylim(bottom=0) # Gap não pode ser negativo
    plt.show()
else:
    print("Não há dados suficientes (custo GRASP e bound MIP válidos) para análise de gap.")

## 5. Análise por Parâmetros ($	au$ e $CV$)

Podemos usar a mesma métrica (Gap do GRASP) para ver se o problema fica mais difícil com o **"Aperto" da Capacidade ($	au$)** ou com a **Variabilidade da Demanda ($var$)**.

In [ ]:
if 'df_gap_analysis' in locals() and len(df_gap_analysis) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))

    # Boxplot por Aperto de Capacidade (tau)
    df_gap_analysis['tau_cat'] = df_gap_analysis['tau'].astype(str)
    sns.boxplot(ax=axes[0], data=df_gap_analysis, x='tau_cat', y='gap_grasp_pct', order=sorted(df_gap_analysis['tau_cat'].unique()))
    axes[0].set_title('Qualidade GRASP vs. Aperto Capacidade ($\tau$)', fontsize=15)
    axes[0].set_xlabel('Aperto ($\tau$)', fontsize=12)
    axes[0].set_ylabel('Gap de Otimalidade do GRASP (%)', fontsize=12)
    axes[0].set_ylim(bottom=0)

    # Boxplot por Variabilidade da Demanda (var)
    df_gap_analysis['var_cat'] = df_gap_analysis['var'].astype(str)
    sns.boxplot(ax=axes[1], data=df_gap_analysis, x='var_cat', y='gap_grasp_pct', order=sorted(df_gap_analysis['var_cat'].unique()))
    axes[1].set_title('Qualidade GRASP vs. Variabilidade Demanda (CV)', fontsize=15)
    axes[1].set_xlabel('Variabilidade (CV)', fontsize=12)
    axes[1].set_ylabel('Gap de Otimalidade do GRASP (%)', fontsize=12)
    axes[1].set_ylim(bottom=0)

    plt.tight_layout()
    plt.show()
else:
    print("Dados de gap não disponíveis para análise por parâmetros.")